In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
MODEL_NAME = "AITeamVN/Vietnamese_Reranker"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# model.resize_token_embeddings(len(tokenizer))  # Ensure the model's token embeddings match the tokenizer size

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

2025-08-26 13:09:57.198362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756213797.370647      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756213797.417508      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [5]:
import pandas as pd
from random import Random

# df_train_src = pd.read_excel("train_data_proptit.xlsx")
# df_doc = pd.read_csv("CLB_PROPTIT.csv")

# queries = []
# article_ids = []
# articles = []
# scores = []

# for index, row in df_train_src.iterrows():
#     query = df_train_src["Query"][index]
#     ground_truth_doc = df_train_src["Ground truth document"][index]
#     ground_truth_docs = []
#     if type(ground_truth_doc) is str:
#         for doc in ground_truth_doc.split(","):
#             ground_truth_docs.append(int(doc) - 1)
#     else:
#         ground_truth_docs.append(int(ground_truth_doc) - 1)
#     for doc_index in ground_truth_docs:
#         queries.append(query)
#         articles.append(df_doc.iloc[doc_index]["Văn bản"])
#         article_ids.append(doc_index + 1)
#         scores.append(1.0)
#     for i in range(2):
#         random_index = Random().randint(0, len(df_doc) - 1)
#         while random_index in ground_truth_docs:
#             random_index = Random().randint(0, len(df_doc) - 1)
#         queries.append(query)
#         articles.append(df_doc.iloc[random_index]["Văn bản"])
#         article_ids.append(random_index + 1)
#         scores.append(0.0)

# df_train = pd.DataFrame({
#     "query": queries,
#     "article": articles,
#     "article_id": article_ids,
#     "score": scores
# })

# df_train.to_csv("finetune_data_proptit.csv", index=False)

In [3]:
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
import torch

df_finetune_data = pd.read_csv("/kaggle/input/finetune-data/finetune_data_proptit.csv")
MODEL_SAVE_PATH = "/kaggle/working/" + MODEL_NAME + "_finetuned"

class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        return item["query"], item["article"], item["score"]

def custom_collate_fn(batch):
    queries, articles, scores = zip(*batch)
    inputs = tokenizer(list(queries), list(articles), return_tensors='pt', padding=True, truncation=True) 
    inputs['labels'] = torch.tensor(scores, dtype=torch.float32)
    return inputs

 
dataset = MyDataset(df_finetune_data)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)

In [4]:
from torch.nn import BCEWithLogitsLoss
from tqdm.notebook import tqdm
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
loss_fn = BCEWithLogitsLoss()
# device = "cpu"
NUM_EPOCHS = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        inputs = {k: batch[k] for k in ['input_ids', 'attention_mask', 'token_type_ids'] if k in batch}
        outputs = model(**inputs)
        logits = outputs.logits
        loss = loss_fn(logits.squeeze(), batch['labels'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} finished. Average Loss: {avg_loss:.4f}")
    model.save_pretrained(MODEL_SAVE_PATH)
    tokenizer.save_pretrained(MODEL_SAVE_PATH)
    print(f"Model saved to {MODEL_SAVE_PATH}")

Epoch 1/15:   0%|          | 0/298 [00:00<?, ?it/s]

Epoch 1 finished. Average Loss: 0.5167
Model saved to /kaggle/working/AITeamVN/Vietnamese_Reranker_finetuned


Epoch 2/15:   0%|          | 0/298 [00:00<?, ?it/s]

Epoch 2 finished. Average Loss: 0.2191
Model saved to /kaggle/working/AITeamVN/Vietnamese_Reranker_finetuned


Epoch 3/15:   0%|          | 0/298 [00:00<?, ?it/s]

...

Epoch 13 finished. Average Loss: 0.0032
Model saved to /kaggle/working/AITeamVN/Vietnamese_Reranker_finetuned


Epoch 14/15:   0%|          | 0/298 [00:00<?, ?it/s]

Epoch 14 finished. Average Loss: 0.0039
Model saved to /kaggle/working/AITeamVN/Vietnamese_Reranker_finetuned


Epoch 15/15:   0%|          | 0/298 [00:00<?, ?it/s]

Epoch 15 finished. Average Loss: 0.0071
Model saved to /kaggle/working/AITeamVN/Vietnamese_Reranker_finetuned
